In [13]:

import urllib.request
import pandas as pd
import requests
import io
import numpy as np
import seaborn as sns
import statsmodels.api as sm
from numpy import mean
from numpy import std
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.linear_model import LogisticRegression
from matplotlib import pyplot
from statsmodels.discrete.discrete_model import MNLogit
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss

hsb2 = pd.read_csv("C:\\Users\\Admin\\Desktop\\Vaishnavi\\Auto\\DriverData1.csv")

hsb2["Gap"] = hsb2["Gap"].astype('int64')
hsb2["RelativeSpeed"] = hsb2["RelativeSpeed"].astype('int64')
hsb2["TW"] = hsb2["TW"].astype('category')
hsb2['Choice']=hsb2['Choice']-1

hsb3 = hsb2
hsb3.head()

,ID,SubVehID,Choice,Gap,RelativeSpeed,TW
0,1,9,1,12,0,0
1,2,13,0,16,2,1
2,3,13,2,15,1,1
3,4,13,0,15,0,1
4,5,36,1,12,0,0


In [14]:
y = hsb3['Choice']
X = hsb3.drop(['ID','SubVehID','Choice'],axis=1)

In [10]:

import pandas as pd
import statsmodels.api as sm
from scipy import stats
from sklearn.metrics import log_loss
# Create a multinomial logistic regression model


model =LogisticRegression(multi_class='multinomial', solver='lbfgs', penalty='l2', 
                           C=1.0, max_iter=1000000)

# Fit the model to the data
model.fit(X, y)
print("Intercept:",model.intercept_)
print("Coefficients:",model.coef_)



# Get the log-likelihood of the fitted model
y_prob = model.predict_proba(X)
LL_model = -log_loss(y, y_prob, normalize=False)

# Fit the null model (only an intercept)
null_model = LogisticRegression(multi_class='multinomial', solver='lbfgs', penalty='l2', 
                                C=1.0, max_iter=1000000, fit_intercept=True)
X_null = np.ones((X.shape[0], 1))  # Only intercept
null_model.fit(X_null, y)

# Get the log-likelihood of the null model
y_null_prob = null_model.predict_proba(X_null)
LL_null = -log_loss(y, y_null_prob, normalize=False)


chi_square_stat = 2 * (LL_model - LL_null)
df_model = X.shape[1] * (len(np.unique(y)) - 1) 
df_null = len(np.unique(y)) - 1  # Number of intercepts in the null model
df_diff = df_model - df_null
p_value = stats.chi2.sf(chi_square_stat, df_diff)
rho_squared = 1 - (LL_model / LL_null)
print(" ")
print("Rho-squared:", rho_squared)
print("Chi-square statistic:", chi_square_stat)
print("p-value:", p_value)


Intercept: [ 2.14744897 -0.45742293 -1.69002604]
Coefficients: [[-0.12946688 -0.20774494  0.15692087]
 [-0.02023765 -0.07849998  0.0538605 ]
 [ 0.14970453  0.28624493 -0.21078137]]
 
Rho-squared: 0.12235023609532403
Chi-square statistic: 577.9456458256163
p-value: 9.184486612176098e-124


In [11]:

c = []
# Initialize an empty list to store predicted probabilities
for i in range(len(hsb2.ID)):
    row = X.iloc[i:i+1, :]  # Select a single row from the DataFrame
    # Predict a multinomial probability distribution
    yhat = model.predict(row)
    # Summarize the predicted probabilities and append to the list
    c.append(yhat[0])
    

hsb2['Predicted'] = c

In [7]:
X.shape

(2400, 3)

In [5]:
import numpy as np

def log_likelihood(X, y, beta):
    """
    Calculate the log likelihood for multinomial logistic regression.
    
    Args:
        X (numpy.ndarray): Design matrix (shape: N x M), where N is the number of samples and M is the number of features.
        y (numpy.ndarray): Target labels (shape: N x 1), where each label corresponds to a class (0, 1, ..., K-1).
        beta (numpy.ndarray): Model coefficients (shape: M x K), where K is the number of classes.
    
    Returns:
        float: Log likelihood value.
    """
    W = np.dot(X, beta)
    eXB = np.exp(W)
    eXB /= eXB.sum(axis=1)[:, None]
    
    log_likelihood_value = np.sum(y * np.log(eXB))
    return log_likelihood_value

# Example usage:
# Assuming you have X (design matrix), y (target labels), and beta (model coefficients)
# log_likelihood_value = log_likelihood(X, y, beta)
# print(f"Log Likelihood: {log_likelihood_value:.4f}")
# Initialize theta with correct shape
num_classes = 3  # Assuming 3 classes
num_features = X.shape[1]
beta = np.random.randn(num_features, num_classes)

ll = log_likelihood(X, y, beta)
print("Log Likelihood:", ll)

ValueError: operands could not be broadcast together with shapes (2400,) (2400,3) 

In [5]:

from scipy import stats
from scipy.stats import chi2


def softmax(z):
    """
    Compute the softmax of each row of the input array z.
    
    Parameters:
    z (ndarray): Input array of shape (n, k) where n is the number of samples and k is the number of classes.
    
    Returns:
    ndarray: Output array of the same shape as z where each row is the softmax of the corresponding row of z.
    """
      # Subtract max for numerical stability
    a=np.exp(z) / np.sum(np.exp(z), axis=1, keepdims=True)
    print(a)
    return a




# Initialize theta with correct shape
num_classes = 3  # Assuming 3 classes
num_features = X.shape[1]
theta = np.random.randn(num_features, num_classes)
print(theta)


ll = softmax(np.dot(X, theta))





[[ 0.19730665  0.51714549  0.5896668 ]
 [-0.98319007 -0.38771223 -1.16472142]
 [-1.12652781 -1.28837583 -1.49074854]]
[[6.31705842e-03 2.93336300e-01 7.00346641e-01]
 [1.37844694e-03 6.43863377e-01 3.54758176e-01]
 [2.52232779e-03 4.71724258e-01 5.25753414e-01]
 ...
 [5.90272631e-03 3.21009664e-01 6.73087610e-01]
 [1.31351750e-03 8.08254314e-01 1.90432169e-01]
 [8.07710612e-04 9.01531063e-01 9.76612262e-02]]


In [20]:

from scipy import stats
from scipy.stats import chi2


def softmax(z):
    """
    Compute the softmax of each row of the input array z.
    
    Parameters:
    z (ndarray): Input array of shape (n, k) where n is the number of samples and k is the number of classes.
    
    Returns:
    ndarray: Output array of the same shape as z where each row is the softmax of the corresponding row of z.
    """
      # Subtract max for numerical stability
    a=np.exp(z) / np.sum(np.exp(z), axis=1, keepdims=True)
    return a

def log_likelihood(params, X, y, num_classes):
    
    # Reshape the parameter array into a coefficient matrix
    beta = params.reshape(num_classes, num_features + 1)
    # Add a column of ones for the intercept term
   
    
    # Compute the linear combination
    logits = np.dot(X, beta.T)
    # Compute the probabilities using the softmax function
    probabilities = softmax(logits)
    # Compute the log-likelihood
    log_likelihood = np.sum(np.log(probabilities[np.arange(num_samples), y]))
    return -log_likelihood  # Return the negative log-likelihood for minimization


num_classes = 3  # Assuming 3 classes
num_samples, num_features = X.shape
theta = np.random.randn(num_features, num_classes)
X_intercept = np.hstack([np.ones((num_samples, 1)), X])
print(theta)
X_null = np.ones((X.shape[0], 3))  # Only intercept

ll = log_likelihood_multinomial_logistic(X, y, theta)
print(ll)

[[-1.02077279  1.19874199  0.83724988 -0.17549782]
 [ 1.47433424 -1.53466741  0.56597923  1.55306486]
 [-1.01193267  0.54309464 -0.65625141 -0.13848144]]
-39217.464098503384


In [23]:

from scipy import stats
from scipy.stats import chi2

def softmax(logits):
    """
    Compute the softmax for a set of logits.

    Parameters:
    logits (np.ndarray): An array of logits with shape (n_samples, n_classes).

    Returns:
    np.ndarray: An array of probabilities with the same shape as logits.
    """
    # Subtract the max logits to prevent overflow issues
    exp_logits = np.exp(logits - np.max(logits, axis=1, keepdims=True))
    # Calculate softmax by dividing the exponentiated logits by the sum of exponentiated logits for each sample
    probabilities = exp_logits / np.sum(exp_logits, axis=1, keepdims=True)
    return probabilities

def log_likelihood_multinomial_logistic(X, y, theta):
    """
    Calculate the log likelihood of the data given a multinomial logistic regression model.
    
    Parameters:
    X (ndarray): Input feature matrix of shape (n, d).
    y (ndarray): True class labels of shape (n,).
    theta (ndarray): Model parameters of shape (d, k).
    
    Returns:
    float: Log likelihood value.
    """
    n, d = X.shape
    k = theta.shape[1]
    
    # Compute the linear combination
    linear_combination = np.dot(X, theta)
    
    # Compute the softmax probabilities
    probabilities = softmax(linear_combination)
    
    # Create a one-hot encoded matrix of the true labels
    y_one_hot = np.zeros((n, k))
    y_one_hot[np.arange(n), y] = 1
    
    # Compute the log likelihood
    log_likelihood = np.sum(y_one_hot * np.log(probabilities))
    
    return log_likelihood

# Example usage


# Initialize theta with correct shape
num_classes = 3  # Assuming 3 classes
num_features = X.shape[1]
theta = np.random.randn(num_features, num_classes)

X_null = np.ones((X.shape[0], 3))  # Only intercept

ll = log_likelihood_multinomial_logistic(X, y, theta)
print("Log Likelihood:", ll)

null_ll = log_likelihood_multinomial_logistic(X_null, y, theta)
print("Null Log Likelihood:", null_ll)

chi_square_stat = 2 * (ll - null_ll)

df_model = X.shape[1] * (len(np.unique(y)) - 1) 
df_null = len(np.unique(y)) - 1  # Number of intercepts in the null model
df_diff = df_model - df_null
p_value = stats.chi2.sf(chi_square_stat, df_diff)
rho_squared = 1 - (ll / null_ll)
print(" ")
print("Rho-squared:", rho_squared)
print("Chi-square statistic:", chi_square_stat)
print("p-value:", p_value)



Log Likelihood: -17244.615875651383
Null Log Likelihood: -4017.108143319909
 
Rho-squared: -3.292793537143787
Chi-square statistic: -26455.01546466295
p-value: 1.0


In [11]:
import numpy as np
from scipy.optimize import minimize
from scipy.special import softmax

# Define the softmax function
def softmax(z):


# Define the log-likelihood function
def log_likelihood(params, X, y, num_classes):
    num_samples, num_features = X.shape
    # Reshape the parameter array into a coefficient matrix
    beta = params.reshape(num_classes, num_features + 1)
    # Add a column of ones for the intercept term
    X_intercept = np.hstack([np.ones((num_samples, 1)), X])
    
    # Compute the linear combination
    logits = np.dot(X_intercept, beta.T)
    # Compute the probabilities using the softmax function
    probabilities = softmax(logits)
    # Compute the log-likelihood
    log_likelihood = np.sum(np.log(probabilities[np.arange(num_samples), y]))
    return -log_likelihood  # Return the negative log-likelihood for minimization

# Define a function to fit the model
def fit_multinomial_logistic_regression(X, y, num_classes, max_iter=1000000):
    num_samples, num_features = X.shape
    # Initialize parameters
    initial_params = np.zeros((num_classes, num_features + 1)).ravel()
    # Optimize the log-likelihood function
    result = minimize(log_likelihood, initial_params, args=(X, y, num_classes),
                      method='L-BFGS-B', options={'maxiter': max_iter})
    # Reshape the result into the coefficient matrix
    beta = result.x.reshape(num_classes, num_features + 1)
    return beta


num_classes = 3

# Subtract 1 from y to ensure class labels start from 0


# Fit the model
beta = fit_multinomial_logistic_regression(X, y, num_classes)


print("Intercept and Coefficients:")
print(beta)




IndentationError: expected an indented block after function definition on line 6 (3100923982.py, line 10)

In [13]:
# Function to predict probabilities
def predict_probabilities(X, beta):
    X_intercept = np.hstack([np.ones((X.shape[0], 1)), X])
    logits = np.dot(X_intercept, beta.T)
    probabilities = softmax(logits)
    return probabilities

# Predict probabilities and class labels
c = []
y_pred = []

for i in range(len(hsb2)):
    row = X.iloc[i:i+1, :]  # Select a single row from the DataFrame
    # Predict a multinomial probability distribution
    phat = predict_probabilities(row, beta)
    # Summarize the predicted probabilities and append to the list
    c.append(phat[0])
    # Determine the predicted class label (the one with the highest probability)
    y_pred.append(np.argmax(phat))
hsb2['Predicted_1'] = y_pred

In [14]:
# Fitting the MNLogit model
X1 = sm.add_constant(X)

model = sm.MNLogit(y, X1)
result = model.fit()

# Display the summary of the model
print(result.summary())


Optimization terminated successfully.
         Current function value: 0.863698
         Iterations 6
                          MNLogit Regression Results                          
Dep. Variable:                 Choice   No. Observations:                 2400
Model:                        MNLogit   Df Residuals:                     2392
Method:                           MLE   Df Model:                            6
Date:                Thu, 13 Jun 2024   Pseudo R-squ.:                  0.1224
Time:                        12:00:50   Log-Likelihood:                -2072.9
converged:                       True   LL-Null:                       -2361.8
Covariance Type:            nonrobust   LLR p-value:                1.331e-121
     Choice=1       coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
const            -2.6055      0.604     -4.311      0.000      -3.790      -1.421
Gap               0.

In [15]:
# Predict probabilities
pred_probs = result.predict(X1)

# Predict the class with the highest probability
y_pred = np.argmax(pred_probs.values, axis=1)

# Add predictions to the dataframe
hsb2['Predicted_Class'] = y_pred



In [16]:
hsb2.to_csv("Results.csv", index=False)

In [30]:
n, d = X.shape
k = theta.shape[1]
    

    
    # Create a one-hot encoded matrix of the true labels
y_one_hot = np.zeros((n, k))
y_one_hot[np.arange(n), y] = 1
print(y_one_hot)

[[0. 1. 0.]
 [1. 0. 0.]
 [0. 0. 1.]
 ...
 [1. 0. 0.]
 [0. 0. 1.]
 [0. 1. 0.]]


ID                  int64
SubVehID            int64
Choice              int64
Gap                 int64
RelativeSpeed       int64
TW               category
dtype: object